<a href="https://colab.research.google.com/github/Jinukki/KJU/blob/master/3minute_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install torch==1.4.0+cpu torch# vision==0.5.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

ERROR: Invalid requirement: 'torch#'


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [0]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [0]:
EPOCHS = 40
BATCH_SIZE = 64

In [5]:
train_loader=torch.utils.data.DataLoader(datasets.FashionMNIST('./.data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])), batch_size=BATCH_SIZE, shuffle=True)
test_loader=torch.utils.data.DataLoader(datasets.FashionMNIST('./.data', train=False, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])), batch_size=BATCH_SIZE, shuffle=True)


Extracting ./.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



Extracting ./.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw



Extracting ./.data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



Extracting ./.data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw
Processing...
Done!


In [0]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
# 1개의 특징맵 받아 10개의 특징맵 생성, 5x5 커널 / * 특성 크기만 넣어줌
    self.conv1 = nn.Conv2d(1,10, kernel_size=5)
# 10개의 특징맵 받아 20개의 특징맵 생성, 5x5 커널
    self.conv2 = nn.Conv2d(10,20, kernel_size=5)
# 드랍아웃 설정
    self.conv2_drop=nn.Dropout2d()
# 일반 신경망, 320과 50의 숫자는 임의로 설정, 10개는 10개의 카테고리 의미
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
# 컨볼루션, 최댓값 풀링 후 relu 활성화 함수 통과
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
# 1차원으로 변환 후 신경망 계층 구성
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    return F.log_softmax(x, dim=1)

In [0]:
# 모델 및 최적화 알고리즘 설정
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [0]:
# 훈련모드 설정 및 진행
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target=data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output=model(data)
        loss=F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(epoch, batch_idx*len(data), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.item()))

In [0]:
# 평가모드로 진행 후 오차 및 정확도 계산
def evaluate(model, test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for data, target in test_loader :
            data, target=data.to(DEVICE), target.to(DEVICE)
            output=model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred=output.max(1,keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy=100.*correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [32]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy=evaluate(model, test_loader)
    
    print('[{}] Test Loss : {:.4f}, Accuracy : {:.2f}%'.format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss:2.361905
Train Epoch: 1 [12800/60000 (21%)]	Loss:1.447812
Train Epoch: 1 [25600/60000 (43%)]	Loss:1.024396
Train Epoch: 1 [38400/60000 (64%)]	Loss:0.835378
Train Epoch: 1 [51200/60000 (85%)]	Loss:0.959103
[1] Test Loss : 0.7034, Accuracy : 72.44%
Train Epoch: 2 [0/60000 (0%)]	Loss:0.740609
Train Epoch: 2 [12800/60000 (21%)]	Loss:0.650407
Train Epoch: 2 [25600/60000 (43%)]	Loss:0.903427
Train Epoch: 2 [38400/60000 (64%)]	Loss:0.876367
Train Epoch: 2 [51200/60000 (85%)]	Loss:0.829937
[2] Test Loss : 0.5859, Accuracy : 77.27%
Train Epoch: 3 [0/60000 (0%)]	Loss:0.794724
Train Epoch: 3 [12800/60000 (21%)]	Loss:0.633185
Train Epoch: 3 [25600/60000 (43%)]	Loss:0.715488
Train Epoch: 3 [38400/60000 (64%)]	Loss:0.534966
Train Epoch: 3 [51200/60000 (85%)]	Loss:0.682119
[3] Test Loss : 0.5295, Accuracy : 79.88%
Train Epoch: 4 [0/60000 (0%)]	Loss:0.677809
Train Epoch: 4 [12800/60000 (21%)]	Loss:0.760174
Train Epoch: 4 [25600/60000 (43%)]	Loss:0.530742
Train Epoch:

In [0]:
EPOCHS = 300
BATCH_SIZE = 128

In [0]:
# CIFAR10 데이터셋 불러오기
train_loader=torch.utils.data.DataLoader(datasets.CIFAR10('./.data', train=True, download=True, transform=transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])), batch_size=BATCH_SIZE, shuffle=True)
test_loader=torch.utils.data.DataLoader(datasets.CIFAR10('./.data', train=False, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])), batch_size=BATCH_SIZE, shuffle=True)

Files already downloaded and verified


In [0]:
# 입력 데이터를 계층을 건너뛰어가며 더해주는 Residual블록을 BasicBlock으로 정의
class BasicBlock(nn.Module):
  def __init__(self, in_planes, planes, stride=1):
    super(BasicBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
# 기울기의 소실이나 발산을 예방하는 배치정규화, 드롭아웃과 같은 효과
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)
# in_planes를 받아 self.bn2 계층의 출력 크기와 같은 planes와 더해주는 shortcut모듈 정의
# nn.Sequential() 은 여러 모듈을 하나의 모듈로 묶는 역할, 여러 모듈을 순서대로 값을 전달해 처리
    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != planes:
      self.shortcut = nn.Sequential(nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False), nn.BatchNorm2d(planes))

  def forward(self, x):
# 입력 x가 컨볼루션, 배치정규화, 활성화 함수 거침
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
# 다시 shortcut 거치게 하고 더해줌
    out += self.shortcut(x)
# 이 값을 relu를 거치게 하여 최종 출력을 얻음
    out = F.relu(out)
    return out

In [0]:
class ResNet(nn.Module):
  def __init__(self, num_classes=10):
    super(ResNet, self).__init__()
# self._make_layer 함수에서 각 층을 만들 때 전 층의 채널 출력값을 기록
# layer1이 입력받는 채널 개수가 16개이므로 16으로 초기화
    self.in_planes = 16
# RGB의 3개의 채널을 16개로 만듦
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(16)
    self.layer1 = self._make_layer(16, 2, stride=1) # 16x32x32
    self.layer2 = self._make_layer(32, 2, stride=2) # 32x16x16
    self.layer3 = self._make_layer(64, 2, stride=2) # 64x8x8
    self.linear = nn.Linear(64, num_classes) # 10개의 값 출력

# _make_layer 함수는 여러 BasickBlock 모듈을 하나로 묶어주는 역할
  def _make_layer(self, planes, num_blocks, stride):
    strides = [stride] + [1]*(num_blocks-1)
# layers 정보를 저장할 딕셔너리 설정
    layers = [] 
    for stride in strides :
# self.in_planes 채널 개수로부터 직접 입력받은 변수인 planes 채널 개수만큼 출력하는 BasicBlock 생성
# layer에 추가
# layer1 : 16채널을 받아 16채널을 내보내는 BasickBlock 2개 strides=[1,1]
# layer2 : 16채널을 받아 32채널 출력하는 것 1개, 32채널에서 32채널 내보내는 것 1개 strides=[1,2]
# layer3 : 32채널을 받아 64채널 출력하는 것 1개, 64채널에서 64채널 내보내는 것 1개 strides=[1,2]
      layers.append(BasicBlock(self.in_planes, planes, stride))
      self.in_planes = planes
# layer 2,3과 같이 증폭하는 모듈들은 shortcut모듈을 따로 가짐
# shortcut 모듈은 이전 입력층을 중간층에 더해주어 이미지의 맥락이 보존될 수 있도록 하는 역할
    return nn.Sequential(*layers)

  def forward(self, x):
# 각 layer는 2개의 Residual 블록을 가짐
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
# 평균풀링을 통해 크기를 줄여줌
    out = F.avg_pool2d(out, 8)
    out = out.view(out.size(0), -1)
    out = self.linear(out)
    return out

In [0]:
model = ResNet().to(DEVICE)
# 학습률 감소기법(최적화 함수의 학습률을 점점 낮춰 더 정교하게 최적화) 사용
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0005)
# 50번 호출될 때 lr*gamma 진행 -> 50이폭 후에 lr = 0.01이 됨
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
# 모델 확인
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [0]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target=data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output=model(data)
        loss=F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

def evaluate(model, test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for data, target in test_loader :
            data, target=data.to(DEVICE), target.to(DEVICE)
            output=model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred=output.max(1,keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy=100.*correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [0]:
# 손실 및 정확도 확인
for epoch in range(1, EPOCHS+1):
# 학습를을 조금 낮춰주는 단계 추가
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy=evaluate(model, test_loader)
    
    print('[{}] Test Loss : {:.4f}, Accuracy : {:.2f}%'.format(epoch, test_loss, test_accuracy))

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1] Test Loss : 1.5427, Accuracy : 46.83%
[2] Test Loss : 1.7651, Accuracy : 47.43%
[3] Test Loss : 1.7712, Accuracy : 49.88%
[4] Test Loss : 1.0150, Accuracy : 66.22%
[5] Test Loss : 1.0075, Accuracy : 67.87%
[6] Test Loss : 0.7721, Accuracy : 73.82%
[7] Test Loss : 1.2351, Accuracy : 63.76%
[8] Test Loss : 0.7708, Accuracy : 75.37%
[9] Test Loss : 0.9135, Accuracy : 70.71%
[10] Test Loss : 0.6893, Accuracy : 77.34%
[11] Test Loss : 0.8461, Accuracy : 71.72%
[12] Test Loss : 0.7527, Accuracy : 75.59%
[13] Test Loss : 0.7243, Accuracy : 75.79%
[14] Test Loss : 0.9880, Accuracy : 69.55%
[15] Test Loss : 0.7559, Accuracy : 74.21%
[16] Test Loss : 1.2384, Accuracy : 62.40%
[17] Test Loss : 0.7874, Accuracy : 75.57%
[18] Test Loss : 0.7546, Accuracy : 76.18%
[19] Test Loss : 0.7548, Accuracy : 74.94%
[20] Test Loss : 0.8405, Accuracy : 74.43%
[21] Test Loss : 0.7496, Accuracy : 76.81%
[22] Test Loss : 0.7138, Accuracy : 76.84%
[23] Test Loss : 0.8085, Accuracy : 73.80%
[24] Test Loss : 0.7